# My attempt at creating a "only time-based" dataset for predicitons.
Perhaps we can mix and match some of the features from this to the other.

In [2]:
#Imports
import pandas as pd
from collections import Counter
import numpy as np
import xgboost as xgb
import datetime

#Initial Data
receivals_raw_df = pd.read_csv("./data/kernel/receivals.csv")
purchase_orders_raw_df = pd.read_csv("./data/kernel/purchase_orders.csv")
materials_raw_df = pd.read_csv("./data/extended/materials.csv")
transportation_raw_df = pd.read_csv("./data/extended/transportation.csv")

prediction_map_raw_df = pd.read_csv("./data/prediction_mapping.csv")

# Part 1: Preparing Data.
We will try different structures for our dataframe. 

### Option 1
In line with our predictions being "per-day" basis, we will create a dataframe where the net-weight per rm_id per day is aggregated, and we will also include entries for every day (even if there are 0 receivals that day). This data structure might help our models since it is much more similar to the output we want to predict, However, it is hard to incorporate info such as purchase order info.

### Option 2
Dataframe on a "per-receival" basis. This is less in-line with our output, but let's us keep more information. We can also use our models to make predictions of receivals and later convert to a per-day basis as a workaround.

## 1.1 Data Cleaning.
Some data-cleanup is required before going with either option.

In [8]:
valid_ids = prediction_map_raw_df["rm_id"].unique()
print("Unique IDs: ", len(valid_ids))

valid_receivals = receivals_raw_df[receivals_raw_df["rm_id"].isin(valid_ids)]
print(len(valid_receivals["rm_id"])- len(receivals_raw_df["rm_id"]), " entries with invalid rm ids dropped")

#Remove duplicates.
drop_duplicates = valid_receivals.drop_duplicates()
print(len(drop_duplicates["rm_id"])- len(valid_receivals["rm_id"]), " duplicates dropped")

#Remove entries where net weight is undefined.
valid_weights = drop_duplicates.dropna(subset=["net_weight"])
print(len(valid_weights["rm_id"])- len(drop_duplicates["rm_id"]), " entries with invalid weights dropped")

#Merge with purchase orders
receival_purchase_order_df = valid_weights.merge(purchase_orders_raw_df, on=['purchase_order_id', 'purchase_order_item_no'], how="left")

#Correct for different weight units
receival_purchase_order_df.loc[receival_purchase_order_df['unit'] == 'PUND', 'net_weight'] *= 0.453592
receival_purchase_order_df.loc[receival_purchase_order_df['unit'].isna(), 'unit'] = 'kg'
receival_purchase_order_df.drop(columns="unit")

#One-hot for month

#Include column for days
receival_purchase_order_df["days_as_datetime"] = pd.to_datetime(receival_purchase_order_df["date_arrival"], errors="coerce", utc=True)
#Define first and last days
first_day = receival_purchase_order_df["days_as_datetime"].min()
last_day = receival_purchase_order_df["days_as_datetime"].max()
#Normalize days since start
receival_purchase_order_df["days_normalized"] = (receival_purchase_order_df["days_as_datetime"]-first_day).dt.days

print("--------------------")
#receival_purchase_order_df.info()
simple_cleaned_df = receival_purchase_order_df.reset_index()

Unique IDs:  203
-57  entries with invalid rm ids dropped
-3  duplicates dropped
-13  entries with invalid weights dropped
--------------------


### 1.2 Data-preparation: Option 1- Per-day, Per-receival
We can make further preparations to our dataset if we only care about each "day".

In [24]:
#Show what columns are in our dataframe
#simple_cleaned_df.info()

#We will group multiple orders together as we will only include receivals per day. That means we can drop or change some rows.
Option1_cleaning = simple_cleaned_df.drop(columns=(
    ["product_id_x", #Messy when grouping receivals per day. Should ideally include?
    #"purchase_order_id", #Messy when grouping
    "purchase_order_item_no", #Messy when grouping
    "receival_item_no", #Could include if summarizing makes sense
    "batch_id", #Very many missing, Could include if summarizing makes sense
    "receival_status", #Instructed that all are valid
    "supplier_id", #Messy when grouping
    "product_id_y", #Should just match first product_id
    "created_date_time", #Unlikely to be relevant, messy when grouping
    "modified_date_time", #Unlikely to be relevant, messy when grouping
    "unit_id", #All should be KG now
    "unit", #All should be KG now
    "status_id", #unclear what this does
    "status", #should be handled in prior code
    "product_version", #Messy when grouping
    "date_arrival", #Redundant with days_as_datetime
    ]))



## Part 2: Feature Engineering- Preparing new data_frame for per_day, per_id.
We make a new dataframe where for we include each day (roghly 20 years) and each id (203) for a total of 203*~7000 rows. Many of these will be zeros, as there are not deliveries every day, but the "format" will match our desired output.

### 2.1 Aggregates:
With the suggested format, we group all receivals per day.

In [ ]:
daily_aggregates = Option1_cleaning.groupby(['rm_id', 'days_normalized']).agg(
    #The total net_weight for that material on that day
    daily_weight=('net_weight', 'sum'),
    #Number of orders
    num_orders=('purchase_order_id', 'count'),

    #General metrics about the orders
    average_orders=('net_weight', 'mean'),
    max_orders=('net_weight', 'max'),
    std_orders=('net_weight', 'std')
).reset_index()
#daily_aggregates = daily_aggregates.fillna({'daily_weight': 0, 'num_orders': 0, 'avg_order_size': 0, 'max_order_size': 0, 'std_order_size': 0}, inplace=True)
daily_aggregates["average_orders"].fillna(0, inplace=True)
daily_aggregates["max_orders"].fillna(0, inplace=True)
daily_aggregates["std_orders"].fillna(0, inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44188 entries, 0 to 44187
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rm_id            44188 non-null  float64
 1   days_normalized  44188 non-null  int64  
 2   daily_weight     44188 non-null  float64
 3   num_orders       44188 non-null  int64  
 4   average_orders   44188 non-null  float64
 5   max_orders       44188 non-null  float64
 6   std_orders       44188 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 2.4 MB
None
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
44183    0.0
44184    0.0
44185    0.0
44186    0.0
44187    0.0
Name: std_orders, Length: 44188, dtype: float64


C:\Users\aksel\AppData\Local\Temp\ipykernel_8808\3324819586.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  daily_aggregates["average_orders"].fillna(0, inplace=True)
C:\Users\aksel\AppData\Local\Temp\ipykernel_8808\3324819586.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

## 2.1 "Common" Features
Some features are decided by dates, and not dependent on rm_id. We create these first to avoid doing this in a loop.

In [ ]:
#Must run pip install holidays
import holidays
#We get all holidays for a relevant year-range:
years = list(range(2004, 2026 + 1))
all_holidays = holidays.Norway(years = years)


print("The first day is: " , first_day)
#We start with finding the range of days
full_range = pd.date_range(start=first_day, end=last_day)
full_range = pd.to_datetime(full_range)
#We want days as 
full_range_days = (full_range-first_day).days
#Remember when we make predictions we should present each day in terms of days since first_day




#Some features are the same for all ID's we create them once to not have to do it many times.
#We should have days normalized, and date
df_for_all_ids = pd.DataFrame({'days_normalized' : full_range_days})
df_for_all_ids["date"] = full_range
#We should get what day of the week it is, since this is likely a good predictor
df_for_all_ids["day_of_week"] = df_for_all_ids["date"].dt.dayofweek
#Weekend is perhaps even stronger predictor: So we create a seperate column for this
df_for_all_ids["is_weekend"]  = df_for_all_ids['day_of_week'].isin([5, 6])
df_for_all_ids["is_weekend"] = df_for_all_ids["is_weekend"].astype(int)
df_for_all_ids["month"] = df_for_all_ids["date"].dt.month

#To filter out holidays, we have to compare the dates. isin() is apparently deprecated for datetime so we convert both to strings.
df_for_all_ids["date_str"] = df_for_all_ids["date"].dt.strftime('%Y-%m-%d')
holiday_str = set(pd.to_datetime(list(all_holidays.keys())).strftime('%Y-%m-%d'))
df_for_all_ids["is_Holiday"] = df_for_all_ids["date_str"].isin(holiday_str)
df_for_all_ids["is_Holiday"] = df_for_all_ids["is_Holiday"].astype(int)
#Verify py printing how many were marked as holidays
print(df_for_all_ids["is_Holiday"].sum(), "days were marked as holidays")



#Time since last purchase or rolling window..
#How do compensate for our predictions being outside our training data (more specificly AFTER). 
#Should cross validation still be usedd or should we prioritze later dates more in a way?

The first day is:  2004-06-15 11:34:00+00:00
237 days were marked as holidays


In [98]:
#To make our dataframe, we make an empty list we can append to.
df_full = []


#For each ID we create fields for all days.
#for i in range(0, len(valid_ids[0:5:1])):
for i in valid_ids:
    #We create sub-group dataframes that we will append to the list.
    #We copy data from the "felles" dataframes
    df_i = df_for_all_ids

    #We include the rm_id
    df_i["rm_id"] = i

    #df_i["rm_id"] = valid_ids[i]

    #For days where we actually have receivals, we get those receivals from the daily_aggregates dataframe
    df_i = df_i.merge(daily_aggregates[daily_aggregates['rm_id'] == i], on=['rm_id', 'days_normalized'], how='left')

    #If there are no receivals in a day, we add a daily weight of 0. as well as zero for the other aggregate metrics.
    df_i["daily_weight"] = df_i["daily_weight"].fillna(0)
    df_i["average_orders"] = df_i["average_orders"].fillna(0)
    df_i["max_orders"] = df_i["max_orders"].fillna(0)
    df_i["std_orders"] = df_i["std_orders"].fillna(0)


    #We create a column for cumulative weight, although I'm not sure it should be included
    df_i['cumulative_weight'] = df_i['daily_weight'].cumsum()

    #Rolling windows. We create 3 rolling windows, one week, one month, and one 100 days.
    df_i["rolling_7"] = df_i['daily_weight'].rolling(window = 7).mean()
    df_i["rolling_30"] = df_i['daily_weight'].rolling(window = 30).mean()
    df_i["rolling_100"] = df_i['daily_weight'].rolling(window = 100).mean()

    #We append to the list and proceed to the next id in the loop
    df_full.append(df_i)

#When all entries (per_id_per_loop) is added, we create a dataframe of the list.
df_final = pd.concat(df_full, ignore_index=True)



MemoryError: Unable to allocate 23.2 MiB for an array with shape (2, 1521079) and data type int64

In [96]:
#Explore data
df_final.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37465 entries, 0 to 37464
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   days_normalized    37465 non-null  int64              
 1   date               37465 non-null  datetime64[ns, UTC]
 2   day_of_week        37465 non-null  int32              
 3   is_weekend         37465 non-null  int64              
 4   month              37465 non-null  int32              
 5   date_str           37465 non-null  object             
 6   is_Holiday         37465 non-null  int64              
 7   rm_id              37465 non-null  int64              
 8   daily_weight       37465 non-null  float64            
 9   num_orders         0 non-null      float64            
 10  average_orders     37465 non-null  float64            
 11  max_orders         37465 non-null  float64            
 12  std_orders         37465 non-null  float64    

In [97]:
#Print to file
df_final.to_csv("cleaned_data/cleaned_df_final_test.csv", index=False)